In [36]:
import requests
import pandas as pd
import datetime
import urllib3
import DB

from mapa import main

urllib3.disable_warnings()



def consulta_ibov():
    r = requests.get('https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjEyMCwiaW5kZXgiOiJJQk9WIiwic2VnbWVudCI6IjIifQ==', verify=False)
    ibov = pd.DataFrame(r.json()['results'])

    ibov['part']=ibov['part'].str.replace(',', '.').astype(float)


    #reductor = float(r.json()['header']['reductor'].replace('.', '').replace(',', '.'))

    ibov['theoricalQty'] = ibov['theoricalQty'].apply(lambda x: x.replace('.', '').replace(',', '.')).astype(float)
    ibov['cod']= ibov['cod'].astype(str)
    
    ibov.loc[0,'reductor']= float(r.json()['header']['reductor'].replace('.', '').replace(',', '.'))

    
    return ibov

def carteira_ibov(df=consulta_ibov()):
    
    
    ibov=df

    ibov = ibov[['cod','part','theoricalQty']]

    ibov['taxa']= list(DB.get_taxa(ticker_name= x,pos=0) for x in ibov['cod'].tolist())

    # ibov['taxa b3'] = list(calculo_b3(taxa=x) for x in ibov['taxa'].tolist())

    # ibov['taxa corretora'] = list(x*0.1 for x in ibov['taxa'].tolist())

    ibov["Analise Peso x Taxa"] = ibov['taxa']*ibov['part']
    ibov["Analise Peso x Taxa"]= ibov["Analise Peso x Taxa"].round(2)
    ibov=ibov.sort_values(by='Analise Peso x Taxa', ascending=False)
    
    # ibov["Analise Doador"] = (ibov['taxa']-ibov['taxa corretora'])*ibov['part']

    # ibov["Analise Tomador"] = (ibov['taxa']+ ibov['taxa corretora'] + ibov['taxa b3'])*ibov['part']



    # ibov.loc[0,"Aluguel Carteira (Doador)"]= round(sum(ibov["Analise Doador"].tolist())/100,2)

    # ibov.loc[0,"Aluguel Carteira (Tomador)"]= round(sum(ibov["Analise Tomador"].tolist())/100,2)
    ibov=ibov.reset_index()
    ibov.loc[0,"Aluguel Carteira"]= round(sum(ibov["Analise Peso x Taxa"].tolist())/100,2)
    ibov['percentual']=(ibov["Analise Peso x Taxa"]/ibov.loc[0,"Aluguel Carteira"])
    ibov['percentual']=ibov['percentual'].round(2)
    
    
    # ibov.to_excel(path+ 'Ibov__aluguel_carteira_'+ str(datetime.date.today())+'.xlsx')
    return ibov


ibov=carteira_ibov()

df=main()

# ibov
# ibov=pd.merge(data.df['taxa_doado'])





C:\Users\JOAO~1.RAM\AppData\Local\Temp/ipykernel_25408/4238251678.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ibov['taxa']= list(DB.get_taxa(ticker_name= x,pos=0) for x in ibov['cod'].tolist())
C:\Users\JOAO~1.RAM\AppData\Local\Temp/ipykernel_25408/4238251678.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ibov["Analise Peso x Taxa"] = ibov['taxa']*ibov['part']
C:\Users\JOAO~1.RAM\AppData\Local\Temp/ipykernel_25408/4238251678.py:44: SettingWithCopyWarning: 
A value is trying to be set on a cop

executing main


In [37]:
ibov = pd.merge(ibov,df,on='cod',how= 'inner')

ibov["Analise Peso x Taxa Doado"] = ibov['taxa_doado']*ibov['part']

ibov["Analise Peso x Taxa Doado"]= ibov["Analise Peso x Taxa Doado"].round(2)

ibov.loc[0,"Aluguel Carteira Kappa"]= round(sum(ibov["Analise Peso x Taxa Doado"].tolist())/100,2)

ibov['percentual kappa']=(ibov["Analise Peso x Taxa Doado"]/ibov.loc[0,"Aluguel Carteira Kappa"])
ibov['percentual kappa']=ibov['percentual kappa'].round(2)

ibov

,index,cod,part,theoricalQty,taxa,Analise Peso x Taxa,Aluguel Carteira,percentual,taxa_doado,Analise Peso x Taxa Doado,Aluguel Carteira Kappa,percentual kappa
0,78,VIVT3,1.071,4.439331e+08,14.81,15.86,1.15,13.79,15.76,16.88,1.09,15.49
1,37,BIDI11,0.881,4.776173e+08,17.09,15.06,NaN,13.10,24.30,21.41,NaN,19.64
2,16,AMER3,0.765,5.460648e+08,12.26,9.38,NaN,8.16,12.17,9.31,NaN,8.54
3,3,CCRO3,0.686,1.115694e+09,8.80,6.04,NaN,5.25,4.09,2.81,NaN,2.58
4,57,BRKM5,0.705,2.646423e+08,8.45,5.96,NaN,5.18,2.46,1.73,NaN,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...
87,4,ECOR3,0.148,3.392379e+08,0.30,0.04,NaN,0.03,0.32,0.05,NaN,0.05
88,9,MRFG3,0.377,3.482340e+08,0.09,0.03,NaN,0.03,0.10,0.04,NaN,0.04
89,33,BRML3,0.348,8.437287e+08,0.10,0.03,NaN,0.03,0.13,0.05,NaN,0.05
90,85,ELET6,0.415,2.429871e+08,0.08,0.03,NaN,0.03,0.15,0.06,NaN,0.06
